## Constants

In [1]:
import sys, os
import pandas as pd
import polars as pl
import numpy as np
import subprocess
import gc
import optuna
from datetime import datetime, timezone
import warnings
import xgboost as xgb
import joblib as jl
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import matthews_corrcoef
from mlflow.models import infer_signature
import mlflow
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold

today = datetime.now(timezone.utc).strftime("%Y_%m_%d")
warnings.filterwarnings("ignore")

from hyper_params import (
    mushroom_tuning_2024_08_06_1722934727_params,
)

SEED = 108
random.seed(SEED)
N_FOLDS = 12
# model
is_tunning = True
try:
    rs = subprocess.check_output("nvidia-smi")
    device = "cuda" if rs is not None else "cpu"
except (
    Exception
):  # this command not being found can raise quite a few different errors depending on the configuration
    print("No Nvidia GPU in system!")
    device = "cpu"

best_params = {
    "device": device,
    "verbosity": 0,
    "objective": "binary:logistic",
}
best_params.update(mushroom_tuning_2024_08_06_1722934727_params)
best_params

{'device': 'cuda',
 'verbosity': 0,
 'objective': 'binary:logistic',
 'tree_method': 'hist',
 'eta': 0.0696294726051571,
 'max_depth': 0,
 'min_child_weight': 1,
 'gamma': 0.044230646284796976,
 'subsample': 0.9405269471473167,
 'colsample_bytree': 0.2999355523666192,
 'lambda': 0.9746051811186938,
 'alpha': 4.210861941737071}

## Prepare data

In [2]:
y_train_pkl = jl.load("../y_train.pkl")
X_train_pkl = jl.load("../X_train.pkl")

print(f"train size: {X_train_pkl.shape}")

train size: (3116945, 294)


## CV

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


clf: xgb.XGBClassifier = xgb.XGBClassifier(
    **best_params,
    n_estimators=4000,
    early_stopping_rounds=50,
    enable_categorical=True,
)

In [4]:
from tqdm import tqdm

gc.collect()
skf = StratifiedKFold(n_splits=N_FOLDS)

y_preds = []
y_trues = []
for train_index, test_index in tqdm(skf.split(X_train_pkl, y_train_pkl)):
    X_train, X_test = X_train_pkl[train_index], X_train_pkl[test_index]
    y_train, y_test = y_train_pkl[train_index], y_train_pkl[test_index]

    clf.fit(X=X_train, y=y_train, eval_set=[(X_test, y_test)])

    y_pred = clf.predict(X_test)
    y_preds.append(y_pred)
    y_trues.append(y_test)

    del X_train, X_test, y_train, y_test, y_pred
    gc.collect()
# Concatenate the predictions and true labels
y_preds_concat = np.concatenate(y_preds)
y_trues_concat = np.concatenate(y_trues)
mcc = matthews_corrcoef(y_trues_concat, y_preds_concat)
print(f"Validation mcc score: {mcc}")
jl.dump(clf, "../clf.pkl")

0it [00:00, ?it/s]

[0]	validation_0-logloss:0.62808
[1]	validation_0-logloss:0.57444
[2]	validation_0-logloss:0.52920
[3]	validation_0-logloss:0.49783
[4]	validation_0-logloss:0.45835
[5]	validation_0-logloss:0.42422
[6]	validation_0-logloss:0.40108
[7]	validation_0-logloss:0.37320
[8]	validation_0-logloss:0.34676
[9]	validation_0-logloss:0.32602
[10]	validation_0-logloss:0.30922
[11]	validation_0-logloss:0.28825
[12]	validation_0-logloss:0.27167
[13]	validation_0-logloss:0.25550
[14]	validation_0-logloss:0.24140
[15]	validation_0-logloss:0.22823
[16]	validation_0-logloss:0.21667
[17]	validation_0-logloss:0.20659
[18]	validation_0-logloss:0.19646
[19]	validation_0-logloss:0.18624
[20]	validation_0-logloss:0.17659
[21]	validation_0-logloss:0.16629
[22]	validation_0-logloss:0.15773
[23]	validation_0-logloss:0.15169
[24]	validation_0-logloss:0.14326
[25]	validation_0-logloss:0.13687
[26]	validation_0-logloss:0.13249
[27]	validation_0-logloss:0.12580
[28]	validation_0-logloss:0.11958
[29]	validation_0-loglos

1it [01:06, 66.38s/it]

[0]	validation_0-logloss:0.62812
[1]	validation_0-logloss:0.57448
[2]	validation_0-logloss:0.52941
[3]	validation_0-logloss:0.49803
[4]	validation_0-logloss:0.45862
[5]	validation_0-logloss:0.42448
[6]	validation_0-logloss:0.40133
[7]	validation_0-logloss:0.37342
[8]	validation_0-logloss:0.34697
[9]	validation_0-logloss:0.32625
[10]	validation_0-logloss:0.30942
[11]	validation_0-logloss:0.28851
[12]	validation_0-logloss:0.27145
[13]	validation_0-logloss:0.25540
[14]	validation_0-logloss:0.24132
[15]	validation_0-logloss:0.22817
[16]	validation_0-logloss:0.21665
[17]	validation_0-logloss:0.20659
[18]	validation_0-logloss:0.19653
[19]	validation_0-logloss:0.18632
[20]	validation_0-logloss:0.17670
[21]	validation_0-logloss:0.16642
[22]	validation_0-logloss:0.15790
[23]	validation_0-logloss:0.15192
[24]	validation_0-logloss:0.14352
[25]	validation_0-logloss:0.13716
[26]	validation_0-logloss:0.13280
[27]	validation_0-logloss:0.12618
[28]	validation_0-logloss:0.11997
[29]	validation_0-loglos

2it [02:10, 65.07s/it]

...

11it [11:42, 64.06s/it]

[0]	validation_0-logloss:0.62817
[1]	validation_0-logloss:0.57455
[2]	validation_0-logloss:0.52933
[3]	validation_0-logloss:0.49795
[4]	validation_0-logloss:0.45851
[5]	validation_0-logloss:0.42436
[6]	validation_0-logloss:0.40119
[7]	validation_0-logloss:0.37328
[8]	validation_0-logloss:0.34687
[9]	validation_0-logloss:0.32615
[10]	validation_0-logloss:0.30932
[11]	validation_0-logloss:0.28836
[12]	validation_0-logloss:0.27128
[13]	validation_0-logloss:0.25517
[14]	validation_0-logloss:0.24105
[15]	validation_0-logloss:0.22791
[16]	validation_0-logloss:0.21637
[17]	validation_0-logloss:0.20631
[18]	validation_0-logloss:0.19624
[19]	validation_0-logloss:0.18601
[20]	validation_0-logloss:0.17637
[21]	validation_0-logloss:0.16608
[22]	validation_0-logloss:0.15754
[23]	validation_0-logloss:0.15151
[24]	validation_0-logloss:0.14307
[25]	validation_0-logloss:0.13670
[26]	validation_0-logloss:0.13233
[27]	validation_0-logloss:0.12567
[28]	validation_0-logloss:0.11945
[29]	validation_0-loglos

12it [12:47, 63.93s/it]


Validation mcc score: 0.9848005518849403


['../clf.pkl']

In [5]:
submit_df = jl.load("../submit_df.pkl")
X_test_pkl = jl.load("../X_test.pkl")
lb = jl.load("../lb.pkl")
y_preds = clf.predict(X_test_pkl)

In [6]:
pred_classes = lb.inverse_transform(y_preds)
submit_df["class"] = pred_classes
submit_df.to_csv("submission.csv", index=False)